Conexão com o banco de dados

In [1]:
import pandas as pd
import psycopg2  
 

# Configurações de conexão
try:
    conn = psycopg2.connect(
        host="localhost",
        database="projeto_semiestruturado",
        port=5432,
        user="postgres",
        password="@renan123"
    )
    print("Conexão estabelecida com sucesso!")
except psycopg2.Error as e:
    print(f"Erro ao conectar ao PostgreSQL: {e}")
        


Conexão estabelecida com sucesso!


Comunicação com o banco de dados

In [2]:
cursor = conn.cursor()


Leitura do arquivo auxiliar

In [4]:
df = pd.read_csv("base_dados/df_auxiliar.csv", encoding='utf-8')

Insere os dados na Tabela Organizacao_Academcica 

In [5]:
df_organizacao_academica = df[['ORGANIZACAO_ACADEMICA']].drop_duplicates()

for _, row in df_organizacao_academica.iterrows():
    try:
        cursor.execute("""
            INSERT INTO ORGANIZACAO_ACADEMICA (ORGANIZACAO_ACADEMICA) 
            VALUES (%s)
            ON CONFLICT (ORGANIZACAO_ACADEMICA) DO NOTHING
        """, (
            row['ORGANIZACAO_ACADEMICA'],
        ))
    except Exception as e:
        print(f"Erro na linha ORG_ACAD {row['ORGANIZACAO_ACADEMICA']}: {e}")
        conn.rollback()

conn.commit()


Insere os dados na Tabela Categoria_IES 

In [ ]:
df_categoria_IES = df[['SIGLA', 'CATEGORIA_DA_IES']].drop_duplicates()

for _, row in df_categoria_IES.iterrows():
    try:
        cursor.execute(""" 
            INSERT INTO CATEGORIA_DA_IES (SIGLA, NOME_CATEGORIA) 
            VALUES (%s, %s)
            ON CONFLICT (SIGLA) DO NOTHING """, (
            row['SIGLA'],
            row['CATEGORIA_DA_IES']
        ))
    except Exception as e:
        print(f"Erro na linha CATEGORIA {row['SIGLA']}: {e}")
        conn.rollback()
conn.commit()


Insere os dados na Tabela Municipio 

In [ ]:
import pandas as pd 

for _, row in df_municipio.iterrows():
    try:
        codigo_municipio_ibge = int(row['CODIGO_MUNICIPIO_IBGE']) if pd.notna(row['CODIGO_MUNICIPIO_IBGE']) else None

        cursor.execute("""
            INSERT INTO MUNICIPIO (CODIGO_MUNICIPIO_IBGE, MUNICIPIO, UF)
            VALUES (%s, %s, %s)
            ON CONFLICT (CODIGO_MUNICIPIO_IBGE) DO UPDATE SET
                MUNICIPIO = EXCLUDED.MUNICIPIO,
                UF = EXCLUDED.UF
        """, (
            codigo_municipio_ibge,
            row['MUNICIPIO'],
            row['UF']
        ))
    except ValueError as ve: 
        print(f"Erro de conversão de tipo na linha MUNICIPIO {row.get('CODIGO_MUNICIPIO_IBGE', 'N/A')}: {ve}")
        conn.rollback()
    except Exception as e:
        print(f"Erro na linha MUNICIPIO {row.get('CODIGO_MUNICIPIO_IBGE', 'N/A')}: {e}")
        conn.rollback()

conn.commit()

Pega os dados do arquivo, e coloca-os na Tabela geral do Banco de dados

In [11]:
for _, row in df.iterrows():
    try:
        cursor.execute("""
            INSERT INTO Instituicoes (
                CODIGO_DA_IES,
                NOME_DA_IES,
                COMUNITARIA,
                CONFESSIONAL,
                FILANTROPICA,
                SITUACAO_IES,
                SIGLA,
                ORGANIZACAO_ACADEMICA,
                CATEGORIA_DA_IES,
                CODIGO_MUNICIPIO_IBGE
            ) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (CODIGO_DA_IES) DO UPDATE SET
                NOME_DA_IES = EXCLUDED.NOME_DA_IES,
                COMUNITARIA = EXCLUDED.COMUNITARIA,
                CONFESSIONAL = EXCLUDED.CONFESSIONAL,
                FILANTROPICA = EXCLUDED.FILANTROPICA,
                SITUACAO_IES = EXCLUDED.SITUACAO_IES,
                SIGLA = EXCLUDED.SIGLA,
                ORGANIZACAO_ACADEMICA = EXCLUDED.ORGANIZACAO_ACADEMICA,
                CATEGORIA_DA_IES = EXCLUDED.CATEGORIA_DA_IES,
                CODIGO_MUNICIPIO_IBGE = EXCLUDED.CODIGO_MUNICIPIO_IBGE
        """, (
            int(row['CODIGO_DA_IES']),
            row['NOME_DA_IES'],
            bool(row['COMUNITARIA']),
            bool(row['CONFESSIONAL']),
            bool(row['FILANTROPICA']),
            row['SITUACAO_IES'],
            row['SIGLA'],
            row['ORGANIZACAO_ACADEMICA'],
            row['CATEGORIA_DA_IES'],
            int(row['CODIGO_MUNICIPIO_IBGE'])
        ))
    except Exception as e:
        print(f"Erro na linha INSTITUICAO {row['CODIGO_DA_IES']}: {e}")
        conn.rollback()
conn.commit()


Teste para ver se foi os dados para o PostgreSQL

In [12]:
cursor.execute("SELECT COUNT(*) FROM Instituicoes")
print("Total de registros:", cursor.fetchone()[0])

Total de registros: 4525
